In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

In [196]:
pd.set_option('future.no_silent_downcasting', True)

#### Getting the data

In [7]:
symbol = input("Enter the symbol you want to perform DCF on: ")

In [8]:
ticker = yf.Ticker(symbol)

In [6]:
selected_case = int(input("""
Choose 3 cases:
Case 1: Optimistic case
Case 2: Base case
Case 3: worst case
Enter your choice (1-3): """))

In [9]:
Balance_sheet = ticker.balance_sheet.transpose()
Income_statement = ticker.income_stmt.transpose()

#### P&L assumption

In [345]:
def forecaster(models: str, year_on_year: bool) -> pd.DataFrame:
    if models == 'Other revenue':
        revenues = pd.DataFrame(Income_statement['Other revenue'])
    elif models == 'Revenues':
        revenues = pd.DataFrame(Income_statement['Operating Revenue']).rename({'Operating Revenue' : 'Revenues'},axis=1)
    revenues.index = revenues.index.year
    revenues = revenues.transpose()

    actual_revenues = actual.copy()
    actual_revenues = pd.concat([actual_revenues, revenues])
    actual_revenues = actual_revenues.transpose()

    actual_revenues['y-o-y growth'] = actual_revenues[models].pct_change()

    if selected_case == 1:
        forecast['y-o-y growth'] = max(actual_revenues['y-o-y growth'])
    elif selected_case == 2:
        forecast['y-o-y growth'] = (actual_revenues['y-o-y growth']).mean()
    elif selected_case == 3:
        forecast['y-o-y growth'] = (actual_revenues['y-o-y growth'].mean()) / 2

    forecast_revenue = forecast.copy()
    forecast_revenue[f'forecast {models}'] = actual_revenues[models].iloc[-1] * (1 + forecast_revenue['y-o-y growth'])    
     
    for i in range(1, len(forecast_revenue)):
        forecast_revenue.loc[forecast_revenue.index.min() + i,f'forecast {models}'] = forecast_revenue.loc[forecast_revenue.index.min() + i -1, f'forecast {models}'] * (1 + forecast_revenue['y-o-y growth'].iloc[0])
    
    return forecast_revenue     

##### Modelling Revenue from sales and services

In [324]:
actual = pd.DataFrame(columns=sorted(list(Income_statement.index.year)))

In [325]:
forecast = pd.DataFrame(columns=[x + max(list(Income_statement.index.year)) for x in range(1,6)]).transpose()

In [326]:
revenues = pd.DataFrame(Income_statement['Operating Revenue']).rename({'Operating Revenue' : 'Revenues'},axis=1)
revenues.index = revenues.index.year
revenues = revenues.transpose()

In [327]:
actual_revenues = actual.copy()
actual_revenues = pd.concat([actual_revenues, revenues])
actual_revenues = actual_revenues.transpose()

In [328]:
actual_revenues['y-o-y growth'] = actual_revenues['Revenues'].pct_change()
actual_revenues

,Revenues,y-o-y growth
2020,143015000000.0,NaN
2021,168088000000.0,0.175317
2022,198270000000.0,0.179561
2023,211915000000.0,0.06882


In [329]:
if selected_case == 1:
    forecast['y-o-y growth'] = max(actual_revenues['y-o-y growth'])
elif selected_case == 2:
    forecast['y-o-y growth'] = (actual_revenues['y-o-y growth']).mean()
elif selected_case == 3:
    forecast['y-o-y growth'] = (actual_revenues['y-o-y growth'].mean()) / 2

In [331]:
forecast_revenue = forecast.copy()
forecast_revenue['forecast Revenue'] = actual_revenues['Revenues'].iloc[-1] * (1 + forecast_revenue['y-o-y growth'])

In [332]:
for i in range(1, len(forecast_revenue)):
    forecast_revenue.loc[forecast_revenue.index.min() + i,'forecast Revenue'] = forecast_revenue.loc[forecast_revenue.index.min() + i -1, 'forecast Revenue'] * (1 + forecast_revenue['y-o-y growth'].iloc[0])
forecast_revenue

,y-o-y growth,forecast Revenue
2024,0.070616,2.268797e+11
2025,0.070616,2.429011e+11
2026,0.070616,2.600539e+11
2027,0.070616,2.784180e+11
2028,0.070616,2.980788e+11


##### Modelling Other revenues


In [307]:
Income_statement['Other revenue'] = Income_statement['Total Revenue'] - Income_statement['Cost Of Revenue']

In [333]:
revenues = pd.DataFrame(Income_statement['Other revenue'])
revenues.index = revenues.index.year
revenues = revenues.transpose()

In [335]:
actual_oth_revenues = actual.copy()
actual_oth_revenues = pd.concat([actual_oth_revenues, revenues])
actual_oth_revenues = actual_oth_revenues.transpose()

In [339]:
actual_oth_revenues['y-o-y growth'] = actual_oth_revenues['Other revenue'].pct_change()
actual_oth_revenues

,Other revenue,y-o-y growth
2020,96937000000.0,NaN
2021,115856000000.0,0.195168
2022,135620000000.0,0.170591
2023,146052000000.0,0.076921


In [340]:
if selected_case == 1:
    forecast['y-o-y growth'] = max(actual_oth_revenues['y-o-y growth'])
elif selected_case == 2:
    forecast['y-o-y growth'] = (actual_oth_revenues['y-o-y growth']).mean()
elif selected_case == 3:
    forecast['y-o-y growth'] = (actual_oth_revenues['y-o-y growth'].mean()) / 2

In [343]:
forecast_oth_revenue = forecast.copy()
forecast_oth_revenue['forecast Other Revenue'] = actual_oth_revenues['Other revenue'].iloc[-1] * (1 + forecast_oth_revenue['y-o-y growth'])

In [344]:
for i in range(1, len(forecast_oth_revenue)):
    forecast_oth_revenue.loc[forecast_oth_revenue.index.min() + i,'forecast Other Revenue'] = forecast_oth_revenue.loc[forecast_oth_revenue.index.min() + i -1, 'forecast Other Revenue'] * (1 + forecast_oth_revenue['y-o-y growth'].iloc[0])
forecast_oth_revenue

,y-o-y growth,forecast Other Revenue
2024,0.07378,1.568277e+11
2025,0.07378,1.683985e+11
2026,0.07378,1.808229e+11
2027,0.07378,1.941640e+11
2028,0.07378,2.084894e+11
